<a href="https://colab.research.google.com/github/seridem06/colab_vozi/blob/vozi/vozi_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar solo dependencias que funcionen en Colab
!pip install pydub librosa

import os
import random
import numpy as np
import librosa
from pydub import AudioSegment
import IPython.display as ipd
from IPython.display import display, HTML, clear_output
import warnings
warnings.filterwarnings('ignore')

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Configurar rutas
base_path = '/content/drive/MyDrive/Reconocimiento_Facial_Dataset/audios'
nivel_001_path = os.path.join(base_path, 'nivel_001')
nivel_002_path = os.path.join(base_path, 'nivel_002')
feedback_path = os.path.join(base_path, 'pronuncia_bien')

# Verificar que los directorios existen
if not os.path.exists(nivel_001_path):
    print(f"Error: No se encuentra el directorio {nivel_001_path}")
if not os.path.exists(nivel_002_path):
    print(f"Error: No se encuentra el directorio {nivel_002_path}")
if not os.path.exists(feedback_path):
    print(f"⚠️ No se encuentra el directorio de feedback: {feedback_path}")

def listar_audios(directorio):
    """Lista todos los archivos m4a en un directorio"""
    archivos = []
    if os.path.exists(directorio):
        for archivo in os.listdir(directorio):
            if archivo.endswith('.m4a'):
                archivos.append(archivo.replace('.m4a', ''))
    return archivos

class JuegoReconocimientoVoz:
    def __init__(self):
        self.estrellas = 0
        self.palabra_actual = None
        self.nivel_actual = 1
        self.oportunidades_restantes = 3
        self.palabras_nivel_actual = []
        self.indice_palabra_actual = 0
        self.juego_terminado = False

        # Cargar listas de palabras
        self.palabras_nivel_001 = listar_audios(nivel_001_path)
        self.palabras_nivel_002 = listar_audios(nivel_002_path)
        self.audios_feedback = listar_audios(feedback_path)

        print(f"🔊 Audios de feedback disponibles: {self.audios_feedback}")

        # Registrar callbacks
        from google.colab import output
        try:
            output.register_callback('notebook.procesar_resultado', self.procesar_resultado)
            output.register_callback('notebook.reiniciar_juego', self.reiniciar_juego)
            output.register_callback('notebook.terminar_juego', self.terminar_juego)
            output.register_callback('notebook.reproducir_audio', self.reproducir_audio_manual)
        except:
            pass

        # Seleccionar 3 palabras aleatorias para empezar
        self.seleccionar_palabras_nivel()

    def seleccionar_palabras_nivel(self):
        """Selecciona 3 palabras aleatorias para el nivel actual"""
        if self.nivel_actual == 1:
            palabras_disponibles = self.palabras_nivel_001.copy()
        else:
            palabras_disponibles = self.palabras_nivel_002.copy()

        # Seleccionar 3 palabras aleatorias únicas
        if len(palabras_disponibles) >= 3:
            self.palabras_nivel_actual = random.sample(palabras_disponibles, 3)
        else:
            self.palabras_nivel_actual = palabras_disponibles.copy()

        self.indice_palabra_actual = 0
        print(f"🎯 Palabras seleccionadas para el nivel {self.nivel_actual}: {self.palabras_nivel_actual}")

    def obtener_siguiente_palabra(self):
        """Obtiene la siguiente palabra del nivel actual"""
        if self.indice_palabra_actual < len(self.palabras_nivel_actual):
            palabra = self.palabras_nivel_actual[self.indice_palabra_actual]
            self.indice_palabra_actual += 1
            return palabra
        else:
            # Si ya se completaron todas las palabras, seleccionar nuevas
            self.seleccionar_palabras_nivel()
            if self.palabras_nivel_actual:
                return self.palabras_nivel_actual[0]
            return None

    def reproducir_audio_feedback(self, tipo_feedback):
        """Reproduce un audio de feedback según el tipo"""
        try:
            # Mapear tipos de feedback a nombres de archivo posibles
            mapeo_archivos = {
                "cerca": ["estas_cerca", "estas cerca", "cerca"],
                "intentalo": ["inténtalo", "intentalo", "intenta", "inténtalo de nuevo"],
                "bien": ["pronuncia_bien", "pronuncia bien", "bien", "tu puedes"],
                "felicidades": ["felicidades", "excelente", "muy bien", "perfecto"],
                "corregir": ["corregir", "no es la palabra", "escucha bien", "incorrecto"]
            }

            # Buscar el archivo correcto
            archivos_posibles = mapeo_archivos.get(tipo_feedback, [])
            archivo_encontrado = None

            for archivo in archivos_posibles:
                ruta = os.path.join(feedback_path, f"{archivo}.m4a")
                if os.path.exists(ruta):
                    archivo_encontrado = archivo
                    break

            if not archivo_encontrado:
                # Si no encuentra el archivo específico, intentar con cualquier archivo de feedback
                if self.audios_feedback:
                    archivo_encontrado = random.choice(self.audios_feedback)
                    ruta = os.path.join(feedback_path, f"{archivo_encontrado}.m4a")
                else:
                    print(f"⚠️ No hay archivos de feedback disponibles en {feedback_path}")
                    return False

            # Reproducir el audio
            audio = AudioSegment.from_file(ruta, format="m4a")
            temp_path = f"/content/feedback_{tipo_feedback}.wav"
            audio.export(temp_path, format="wav")

            print(f"🎧 Reproduciendo feedback: {archivo_encontrado}")
            audio_widget = ipd.Audio(temp_path, autoplay=True)
            display(audio_widget)
            return True

        except Exception as e:
            print(f"❌ Error reproduciendo feedback {tipo_feedback}: {e}")
            return False

    def reproducir_audio_manual(self):
        """Reproduce el audio actual manualmente"""
        self.reproducir_audio_automatico()

    def mostrar_widget_juego(self):
        """Muestra el widget principal del juego"""
        clear_output(wait=True)

        if self.juego_terminado:
            self.mostrar_final_juego()
            return

        # Widget principal compacto
        widget_html = f'''
        <div style="
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 20px;
            border-radius: 15px;
            color: white;
            font-family: Arial, sans-serif;
            max-width: 500px;
            margin: 20px auto;
            box-shadow: 0 8px 25px rgba(0,0,0,0.3);
        ">
            <!-- Encabezado -->
            <div style="text-align: center; margin-bottom: 20px;">
                <h2 style="margin: 0; font-size: 24px;">🎤 JUEGO DE VOZ</h2>
                <div style="display: flex; justify-content: center; gap: 20px; margin-top: 10px;">
                    <span style="background: rgba(255,255,255,0.2); padding: 5px 10px; border-radius: 10px;">
                        🎯 Nivel {self.nivel_actual}
                    </span>
                    <span style="background: rgba(255,255,255,0.2); padding: 5px 10px; border-radius: 10px;">
                        ⭐ {self.estrellas}/3
                    </span>
                </div>
            </div>

            <!-- Información de la palabra -->
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px; margin-bottom: 15px;">
                <div style="text-align: center;">
                    <h3 style="margin: 0 0 10px 0; color: #ffeb3b;">PALABRA ACTUAL</h3>
                    <div style="font-size: 28px; font-weight: bold; letter-spacing: 2px; margin: 10px 0;">
                        {self.palabra_actual.upper() if self.palabra_actual else "CARGANDO..."}
                    </div>
                    <div style="color: #ffcc80; margin: 5px 0;">
                        Oportunidades: {self.oportunidades_restantes} 💡
                    </div>
                </div>
            </div>

            <!-- Progreso -->
            <div style="background: rgba(255,255,255,0.1); padding: 10px; border-radius: 10px; margin-bottom: 15px;">
                <div style="text-align: center; font-size: 14px;">
                    <strong>Progreso:</strong> {self.indice_palabra_actual}/{len(self.palabras_nivel_actual)} palabras
                </div>
                <div style="display: flex; justify-content: center; gap: 5px; margin-top: 8px;">
                    {"".join(['<span style="color: green;">✓</span>' if i < self.indice_palabra_actual - 1 else
                             '<span style="color: orange;">●</span>' if i == self.indice_palabra_actual - 1 else
                             '<span style="color: gray;">○</span>' for i in range(len(self.palabras_nivel_actual))])}
                </div>
            </div>

            <!-- Botones de control -->
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px; margin-bottom: 15px;">
                <button id="btnReconocer" style="
                    background: linear-gradient(45deg, #FF416C, #FF4B2B);
                    border: none;
                    color: white;
                    padding: 12px;
                    border-radius: 25px;
                    font-weight: bold;
                    cursor: pointer;
                    font-size: 16px;
                    transition: all 0.3s;
                " onmouseover="this.style.transform='scale(1.05)'" onmouseout="this.style.transform='scale(1)'">
                    🎤 HABLAR
                </button>

                <button id="btnReproducir" style="
                    background: linear-gradient(45deg, #2196F3, #21CBF3);
                    border: none;
                    color: white;
                    padding: 12px;
                    border-radius: 25px;
                    font-weight: bold;
                    cursor: pointer;
                    font-size: 16px;
                    transition: all 0.3s;
                " onmouseover="this.style.transform='scale(1.05)'" onmouseout="this.style.transform='scale(1)'">
                    🔄 REPETIR
                </button>
            </div>

            <!-- Botones secundarios -->
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 8px;">
                <button id="btnReiniciar" style="
                    background: #FF9800;
                    border: none;
                    color: white;
                    padding: 8px;
                    border-radius: 15px;
                    cursor: pointer;
                    font-size: 14px;
                ">
                    🔄 Reiniciar
                </button>

                <button id="btnTerminar" style="
                    background: #F44336;
                    border: none;
                    color: white;
                    padding: 8px;
                    border-radius: 15px;
                    cursor: pointer;
                    font-size: 14px;
                ">
                    ❌ Salir
                </button>
            </div>

            <!-- Estado del reconocimiento -->
            <div id="estadoReconocimiento" style="
                margin-top: 15px;
                padding: 10px;
                background: rgba(0,0,0,0.3);
                border-radius: 8px;
                text-align: center;
                font-size: 14px;
                min-height: 20px;
            ">
                👆 Presiona "HABLAR" para empezar
            </div>

            <!-- Resultado -->
            <div id="resultado" style="
                margin-top: 10px;
                padding: 10px;
                border-radius: 8px;
                text-align: center;
                font-size: 14px;
                min-height: 20px;
            "></div>
        </div>
        '''

        display(HTML(widget_html))

        # JavaScript para el reconocimiento de voz
        js_code = f'''
        const palabraObjetivo = "{self.palabra_actual}";
        let reconocimiento;

        // Configurar reconocimiento de voz
        function configurarReconocimiento() {{
            if ('webkitSpeechRecognition' in window || 'SpeechRecognition' in window) {{
                const SpeechRecognition = window.SpeechRecognition || window.webkitSpeechRecognition;
                reconocimiento = new SpeechRecognition();

                reconocimiento.continuous = false;
                reconocimiento.interimResults = false;
                reconocimiento.lang = 'es-ES';

                reconocimiento.onstart = function() {{
                    document.getElementById('estadoReconocimiento').innerHTML = '<span style="color: #FF5252;">● ESCUCHANDO... Habla ahora</span>';
                    document.getElementById('btnReconocer').innerHTML = '🎤 ESCUCHANDO...';
                    document.getElementById('btnReconocer').style.background = 'linear-gradient(45deg, #FF5252, #B71C1C)';
                    document.getElementById('btnReconocer').disabled = true;
                }};

                reconocimiento.onresult = function(event) {{
                    const textoReconocido = event.results[0][0].transcript.toLowerCase().trim();
                    document.getElementById('estadoReconocimiento').innerHTML = '<span style="color: #69F0AE;">✅ Reconocido: ' + textoReconocido + '</span>';

                    // Enviar a Python
                    google.colab.kernel.invokeFunction('notebook.procesar_resultado', [textoReconocido], {{}});
                }};

                reconocimiento.onerror = function(event) {{
                    let mensajeError = '❌ Error: ';
                    if (event.error == 'no-speech') {{
                        mensajeError += 'No se detectó voz';
                    }} else if (event.error == 'audio-capture') {{
                        mensajeError += 'No se pudo capturar audio';
                    }} else if (event.error == 'not-allowed') {{
                        mensajeError += 'Permiso de micrófono denegado';
                    }} else {{
                        mensajeError += event.error;
                    }}

                    document.getElementById('estadoReconocimiento').innerHTML = '<span style="color: #FF5252;">' + mensajeError + '</span>';
                    resetearBotonReconocer();
                }};

                reconocimiento.onend = function() {{
                    resetearBotonReconocer();
                }};

            }} else {{
                document.getElementById('estadoReconocimiento').innerHTML = '<span style="color: #FF5252;">❌ Navegador no compatible con reconocimiento de voz</span>';
                document.getElementById('btnReconocer').disabled = true;
                document.getElementById('btnReconocer').style.background = '#9E9E9E';
            }}
        }}

        function resetearBotonReconocer() {{
            document.getElementById('btnReconocer').innerHTML = '🎤 HABLAR';
            document.getElementById('btnReconocer').style.background = 'linear-gradient(45deg, #FF416C, #FF4B2B)';
            document.getElementById('btnReconocer').disabled = false;
        }}

        // Event listeners
        document.getElementById('btnReconocer').addEventListener('click', function() {{
            try {{
                configurarReconocimiento();
                reconocimiento.start();
            }} catch (error) {{
                document.getElementById('estadoReconocimiento').innerHTML = '<span style="color: #FF5252;">❌ Error al iniciar reconocimiento</span>';
            }}
        }});

        document.getElementById('btnReproducir').addEventListener('click', function() {{
            document.getElementById('estadoReconocimiento').innerHTML = '<span style="color: #2196F3;">🔊 Reproduciendo audio...</span>';
            google.colab.kernel.invokeFunction('notebook.reproducir_audio', [], {{}});
        }});

        document.getElementById('btnReiniciar').addEventListener('click', function() {{
            if(confirm('¿Estás seguro de que quieres reiniciar el juego?')) {{
                google.colab.kernel.invokeFunction('notebook.reiniciar_juego', [], {{}});
            }}
        }});

        document.getElementById('btnTerminar').addEventListener('click', function() {{
            if(confirm('¿Estás seguro de que quieres terminar el juego?')) {{
                google.colab.kernel.invokeFunction('notebook.terminar_juego', [], {{}});
            }}
        }});

        // Inicializar reconocimiento
        configurarReconocimiento();
        '''

        display(ipd.Javascript(js_code))

        # Reproducir audio automáticamente al mostrar el widget
        self.reproducir_audio_automatico()

    def reproducir_audio_automatico(self):
        """Reproduce el audio de referencia automáticamente"""
        try:
            if self.palabra_actual:
                if self.nivel_actual == 1:
                    ruta = os.path.join(nivel_001_path, f"{self.palabra_actual}.m4a")
                else:
                    ruta = os.path.join(nivel_002_path, f"{self.palabra_actual}.m4a")

                if os.path.exists(ruta):
                    audio = AudioSegment.from_file(ruta, format="m4a")
                    temp_path = "/content/audio_actual.wav"
                    audio.export(temp_path, format="wav")

                    print("🎧 Reproduciendo audio de la palabra...")
                    audio_widget = ipd.Audio(temp_path, autoplay=True)
                    display(audio_widget)

        except Exception as e:
            print(f"❌ Error reproduciendo audio: {e}")

    def detectar_confusion_r_l(self, texto_reconocido, palabra_objetivo):
        """Detecta específicamente confusiones entre R y L"""
        texto = texto_reconocido.lower()
        objetivo = palabra_objetivo.lower()

        # Caso 1: Palabra objetivo tiene R pero niño dijo L
        if 'r' in objetivo and 'l' in texto:
            texto_corregido = texto.replace('l', 'r')
            if texto_corregido == objetivo:
                return True, "r_por_l", "Confundiste 'R' con 'L'. Para la R, haz vibrar la lengua"

        # Caso 2: Palabra objetivo tiene RR pero niño dijo L
        if 'rr' in objetivo and 'l' in texto:
            texto_corregido = texto.replace('l', 'rr')
            if texto_corregido == objetivo:
                return True, "rr_por_l", "Confundiste 'RR' con 'L'. Para la RR, vibra la lengua más fuerte"

        # Caso 3: Palabra objetivo tiene L pero niño dijo R
        if 'l' in objetivo and 'r' in texto:
            texto_corregido = texto.replace('r', 'l')
            if texto_corregido == objetivo:
                return True, "l_por_r", "Confundiste 'L' con 'R'. Para la L, toca el paladar con la punta de la lengua"

        return False, "", ""

    def calcular_similitud(self, texto1, texto2):
        """Calcula qué tan similares son dos textos"""
        from difflib import SequenceMatcher
        return SequenceMatcher(None, texto1.lower(), texto2.lower()).ratio()

    def comparar_palabras(self, texto_reconocido, palabra_objetivo):
        """Compara el texto reconocido con la palabra objetivo - SOLO EXACTO"""
        texto_limpio = texto_reconocido.lower().strip()
        objetivo_limpio = palabra_objetivo.lower().strip()

        print(f"🔍 Comparando EXACTO: '{texto_limpio}' con '{objetivo_limpio}'")

        # SOLO COMPARACIÓN EXACTA
        if texto_limpio == objetivo_limpio:
            return True, "exacta", ""

        # Verificar confusiones específicas R/L
        tiene_confusion_rl, tipo_confusion, mensaje_confusion = self.detectar_confusion_r_l(texto_limpio, objetivo_limpio)
        if tiene_confusion_rl:
            return False, tipo_confusion, mensaje_confusion

        # Calcular similitud para determinar el tipo de feedback
        similitud = self.calcular_similitud(texto_limpio, objetivo_limpio)

        if similitud < 0.3:
            return False, "completamente_diferente", ""
        elif similitud < 0.6:
            return False, "poco_similar", ""
        else:
            return False, "muy_similar", ""

    def procesar_resultado(self, texto_reconocido):
        """Procesa el resultado del reconocimiento de voz"""
        import time

        print(f"🎯 Texto reconocido: '{texto_reconocido}'")
        print(f"🎯 Palabra objetivo: '{self.palabra_actual}'")

        # Comparar las palabras - SOLO EXACTO
        es_correcto, tipo_comparacion, mensaje_especifico = self.comparar_palabras(texto_reconocido, self.palabra_actual)

        if es_correcto:
            self.estrellas += 1
            self.oportunidades_restantes = 3

            # Reproducir audio de felicitaciones
            print("✅ ¡ACERTASTE EXACTAMENTE!")
            self.reproducir_audio_feedback("felicidades")

            # Verificar si pasó al nivel 2 o terminó el juego
            if self.estrellas >= 3:
                if self.nivel_actual == 1:
                    print("🚀 Pasando al NIVEL 2...")
                    self.nivel_actual = 2
                    self.estrellas = 0
                    self.seleccionar_palabras_nivel()
                else:
                    self.juego_terminado = True

            time.sleep(3)
            if not self.juego_terminado:
                self.siguiente_palabra()
            else:
                self.mostrar_final_juego()

        else:
            self.oportunidades_restantes -= 1

            # Determinar qué tipo de feedback reproducir
            if tipo_comparacion in ["r_por_l", "rr_por_l", "l_por_r"]:
                self.reproducir_audio_feedback("corregir")
            elif tipo_comparacion == "completamente_diferente":
                self.reproducir_audio_feedback("corregir")
            elif tipo_comparacion == "poco_similar":
                self.reproducir_audio_feedback("bien")
            else:
                if self.oportunidades_restantes == 2:
                    self.reproducir_audio_feedback("cerca")
                elif self.oportunidades_restantes == 1:
                    self.reproducir_audio_feedback("intentalo")
                else:
                    self.reproducir_audio_feedback("bien")

            if self.oportunidades_restantes > 0:
                time.sleep(3)
                self.mostrar_widget_juego()
            else:
                self.oportunidades_restantes = 3
                time.sleep(2)
                self.siguiente_palabra()

    def siguiente_palabra(self):
        """Avanza a la siguiente palabra"""
        if self.juego_terminado:
            self.mostrar_final_juego()
            return

        siguiente_palabra = self.obtener_siguiente_palabra()

        if siguiente_palabra:
            self.palabra_actual = siguiente_palabra
            self.mostrar_widget_juego()
        else:
            self.seleccionar_palabras_nivel()
            if self.palabras_nivel_actual:
                self.palabra_actual = self.palabras_nivel_actual[0]
                self.mostrar_widget_juego()

    def mostrar_final_juego(self):
        """Muestra la pantalla final del juego"""
        clear_output()

        final_html = '''
        <div style="
            background: linear-gradient(135deg, #4CAF50, #45a049);
            padding: 30px;
            border-radius: 20px;
            color: white;
            font-family: Arial, sans-serif;
            max-width: 500px;
            margin: 20px auto;
            text-align: center;
            box-shadow: 0 10px 30px rgba(0,0,0,0.3);
        ">
            <div style="font-size: 48px; margin-bottom: 20px;">🏆</div>
            <h1 style="margin: 0 0 15px 0; font-size: 32px;">¡FELICIDADES!</h1>
            <p style="font-size: 18px; margin-bottom: 25px;">Has completado todos los niveles del juego</p>

            <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px; margin-bottom: 25px;">
                <h3 style="margin: 0 0 10px 0;">🎯 Logros obtenidos:</h3>
                <p style="margin: 5px 0;">✅ Nivel 1: Pronunciación básica</p>
                <p style="margin: 5px 0;">✅ Nivel 2: Pronunciación avanzada</p>
                <p style="margin: 5px 0;">⭐ Dominio completo del reconocimiento de voz</p>
            </div>

            <button onclick="google.colab.kernel.invokeFunction('notebook.reiniciar_juego', [], {})"
                    style="background: #FF9800; color: white; border: none; padding: 15px 30px;
                           border-radius: 25px; font-size: 18px; font-weight: bold; cursor: pointer;
                           margin: 10px; transition: all 0.3s;"
                    onmouseover="this.style.transform='scale(1.05)'"
                    onmouseout="this.style.transform='scale(1)'">
                🔄 Jugar de Nuevo
            </button>
        </div>
        '''

        display(HTML(final_html))

    def reiniciar_juego(self):
        """Reinicia el juego completamente"""
        self.estrellas = 0
        self.nivel_actual = 1
        self.oportunidades_restantes = 3
        self.juego_terminado = False
        self.seleccionar_palabras_nivel()
        if self.palabras_nivel_actual:
            self.palabra_actual = self.palabras_nivel_actual[0]
            self.mostrar_widget_juego()

    def terminar_juego(self):
        """Termina el juego"""
        clear_output()

        mensaje_html = '''
        <div style="
            background: linear-gradient(135deg, #F44336, #D32F2F);
            padding: 30px;
            border-radius: 20px;
            color: white;
            font-family: Arial, sans-serif;
            max-width: 500px;
            margin: 20px auto;
            text-align: center;
        ">
            <h2>🎮 Juego Terminado</h2>
            <p>¡Gracias por jugar!</p>
            <button onclick="google.colab.kernel.invokeFunction('notebook.reiniciar_juego', [], {})"
                    style="background: #4CAF50; color: white; border: none; padding: 10px 20px;
                           border-radius: 15px; cursor: pointer; margin-top: 15px;">
                🔄 Jugar Otra Vez
            </button>
        </div>
        '''

        display(HTML(mensaje_html))

    def iniciar_juego(self):
        """Inicia el juego principal"""
        clear_output()

        # Mostrar instrucciones breves
        instrucciones_html = '''
        <div style="
            background: linear-gradient(135deg, #2196F3, #1976D2);
            padding: 20px;
            border-radius: 15px;
            color: white;
            font-family: Arial, sans-serif;
            max-width: 500px;
            margin: 20px auto;
        ">
            <h3 style="text-align: center; margin-top: 0;">🎤 INSTRUCCIONES RÁPIDAS</h3>
            <ul style="text-align: left; font-size: 14px;">
                <li>🎧 Escucha la palabra que se reproduce</li>
                <li>🎤 Haz clic en "HABLAR" y pronuncia la palabra</li>
                <li>⭐ Gana 3 estrellas para pasar al siguiente nivel</li>
                <li>💡 Tienes 3 oportunidades por palabra</li>
                <li>🔊 La pronunciación debe ser EXACTA</li>
            </ul>
        </div>
        '''

        display(HTML(instrucciones_html))

        self.palabra_actual = self.obtener_siguiente_palabra()
        if self.palabra_actual:
            self.mostrar_widget_juego()
        else:
            print("❌ No se encontraron palabras para jugar")

# Inicializar y ejecutar el juego
print("🔍 Inicializando juego de reconocimiento de voz...")
palabras_nivel_001 = listar_audios(nivel_001_path)
palabras_nivel_002 = listar_audios(nivel_002_path)
audios_feedback = listar_audios(feedback_path)

print(f"📁 Palabras cargadas: N1={len(palabras_nivel_001)}, N2={len(palabras_nivel_002)}")
print(f"📁 Audios de feedback: {len(audios_feedback)}")

if palabras_nivel_001 or palabras_nivel_002:
    juego = JuegoReconocimientoVoz()
    juego.iniciar_juego()
else:
    print("❌ No se encontraron archivos de audio en los directorios")

<IPython.core.display.Javascript object>

🎧 Reproduciendo audio de la palabra...
